In [11]:
import requests
import pandas as pd
import json
import os
from collections import Counter
import numpy as np

In [12]:
cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg",
          "Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon",
          "Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes",
          "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse",
          "Montauban","Biarritz","Bayonne","La Rochelle"]

In [13]:
# Test 

city_name = 'Bayeux,FR'
API_key= "2f2685efc16594e365e34e00af802902"

def get_weather(API_key, city, return_=False):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_key}"
    r = requests.get(url).json()
    print(f"Ville :\n {r['name']}\n")
    print(f"Coordonnées GPS :\n Longitude : {r['coord']['lon']}\n Latitude : {r['coord']['lat']}\n")
    print(f"Conditions Météorologiques :\n Temperature(C°) : {int(r['main']['temp'] - 273.15)} \n Ressenti(C°) : {int(r['main']['feels_like'] - 273.15)} \n Humidité : {r['main']['humidity']}")
    print(f" Météo : {r['weather'][0]['main']}")
    if return_:
        return r

In [14]:
get_weather(API_key, city_name)

Ville :
 Arrondissement de Bayeux

Coordonnées GPS :
 Longitude : -0.8333
 Latitude : 49.25

Conditions Météorologiques :
 Temperature(C°) : 11 
 Ressenti(C°) : 10 
 Humidité : 72
 Météo : Clouds


In [15]:
def create_coords(cities):
    s = []
    url = 'https://nominatim.openstreetmap.org/search'
    cols = ['city_id' , 'name', 'latitude', 'longitude']
    n = len(cities)
    for i in range(n):
        params = {"city": cities[i], "country": "France", "format": "json"}
        r = requests.get(url, params=params).json()
        s.append(r)
        
    for i in range(len(s)):
        if s[i] == []:
            print(cities[i], 'not in the list')
            
    cities_id = [i for i in range(n) if s[i] != []]
    cities_name = [city for i,city in enumerate(cities) if s[i] != []]
    cities_lat = [s[i][0]['lat'] for i in range(n) if s[i] != []]
    cities_lon = [s[i][0]['lon'] for i in range(n) if s[i] != []]
    
    rows = [cities_id, cities_name, cities_lat, cities_lon]  
    df = pd.DataFrame({cols[0] : cities_id, cols[1] : cities_name, cols[2] : cities_lat, cols[3] : cities_lon})
    
    return df

In [16]:
df = create_coords(cities)
df.head()

Gorges du Verdon not in the list
Ariege not in the list


,city_id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


In [17]:
df_temp_ = pd.DataFrame({'city_id' : [17,29], 'name' : ['Gorges du Verdon', 'Ariege'], 'latitude' : [43.7496562, 42.9455368],
                         'longitude' : [6.3285616, 1.4065544]})

df = pd.concat([df,df_temp_], axis=0).sort_values(by='city_id')
df.head()

,city_id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


In [18]:
df.shape

(35, 4)

In [21]:
df.to_csv('kayak2_df.csv', index=None)

In [19]:
df = pd.read_csv('kayak2_df.csv')
df.head()

,city_id,name,latitude,longitude
0,0,Mont Saint Michel,48.635954,-1.511460
1,1,St Malo,48.649518,-2.026041
2,2,Bayeux,49.276462,-0.702474
3,3,Le Havre,49.493898,0.107973
4,4,Rouen,49.440459,1.093966


In [20]:
def get_weather_next_seven_days(df, API_key, coef_temp=2, coef_proba_rain=0.6, coef_himidity=0.2, coef_wind=0.9):
    
    from datetime import datetime
    now = datetime.now()
    temperature, rains, humidity, winds, pressures, descs = [], [], [], [], [], []
    temp_df = pd.DataFrame()
    url = "https://api.openweathermap.org/data/2.5/onecall"
    
    for i in range(len(df)):
        
        params = {
            "lat": df.loc[i,'latitude'],
            "lon": df.loc[i,'longitude'],
            "exclude": "current, minutely, hourly", # Exclusion de certains parametres de  l'API: current, minutely and hourly
            "units": "metric", 
            "appid": "2f2685efc16594e365e34e00af802902", # API key
        }
        
        r = requests.get(url,params).json()
        
        seven_next_days = r['daily'][1:] 
        
        feels = [int(weather['feels_like']['day']) for weather in seven_next_days] # temperature pour les 7 prochains jours
        rain = [int(weather['pop'] * 100) for weather in seven_next_days]     # pourcentage de pluie pour les 7 prochains jours
        humid = [weather['humidity'] for weather in seven_next_days]          # pourcentage d'humidité pour les 7 prochains jours
        wind = [weather['wind_speed'] for weather in seven_next_days]         # vitesse du vent
        pressure = [weather['pressure'] for weather in seven_next_days]       # pression atmospherique
        desc = [weather['weather'][0]['main'] for weather in seven_next_days] # description des conditions meteos
        
        temperature.append(feels)
        rains.append(rain)    
        humidity.append(humid)
        winds.append(wind)
        pressures.append(pressure)
        descs.append(desc)
        
    temp_df['days'] = [[i for i in range(1, 8)] for row in range(len(df))]
    
    temp_df['temperature'] = temperature
    temp_df['mean_temp'] = temp_df['temperature'].apply(lambda x: int(np.array(x).mean()))
    temp_df['proba_rain'] = rains
    temp_df['mean_rain'] = temp_df['proba_rain'].apply(lambda x: np.round(np.array(x).mean()),2)
    temp_df['humidity'] = humidity
    temp_df['mean_humidity'] = temp_df['humidity'].apply(lambda x: np.round(np.array(x).mean()),2)
    temp_df['wind_speed'] = winds
    temp_df['mean_wind_speed'] = temp_df['wind_speed'].apply(lambda x: np.round(np.array(x).mean()),2)
    temp_df['pressure'] = pressures
    temp_df['mean_pressure'] = temp_df['pressure'].apply(lambda x: int(np.array(x).mean()))
    temp_df['weather'] = descs
    temp_df['mode_weather'] = temp_df['weather'].apply(lambda x:pd.Series(x).mode())
    
    mean_temp = [sum(el)/7 for el in temp_df['temperature']]
    mean_prob_rain = [sum(el)/7 for el in temp_df['proba_rain']]
    mean_humid = [sum(el)/7 for el in temp_df['humidity']]
    mean_wind = [sum(el)/7 for el in temp_df['wind_speed']]
    
    score = np.round((np.array(mean_temp)*coef_temp + np.array(mean_prob_rain)*coef_proba_rain + np.array(mean_humid)*coef_himidity + np.array(mean_wind)*coef_wind) / 3,2) 
    
    temp_df['score'] = score  
    temp_df['date'] = now.strftime("%m/%d/%Y, %H:%M:%S")
    
    return pd.concat([df,temp_df],axis=1)

In [21]:
df_ = get_weather_next_seven_days(df, API_key)
df_.head()

,city_id,name,latitude,longitude,days,temperature,mean_temp,proba_rain,mean_rain,humidity,mean_humidity,wind_speed,mean_wind_speed,pressure,mean_pressure,weather,mode_weather,score,date
0,0,Mont Saint Michel,48.635954,-1.511460,"[1, 2, 3, 4, 5, 6, 7]","[10, 10, 4, 10, 11, 12, 14]",10,"[32, 87, 100, 87, 100, 100, 100]",87.0,"[68, 65, 94, 69, 73, 86, 75]",76.0,"[3.89, 9.2, 13.33, 8.96, 10.75, 10.93, 9.38]",9.0,"[1016, 1014, 995, 1009, 1005, 1011, 1009]",1008,"[Clear, Rain, Rain, Rain, Rain, Rain, Rain]",Rain,31.97,"11/18/2022, 13:07:14"
1,1,St Malo,48.649518,-2.026041,"[1, 2, 3, 4, 5, 6, 7]","[10, 11, 10, 10, 11, 10, 13]",10,"[40, 100, 100, 100, 100, 100, 100]",91.0,"[58, 63, 93, 67, 73, 74, 74]",72.0,"[5.62, 10.23, 17.16, 13.46, 12.86, 14.61, 12.93]",12.0,"[1016, 1015, 986, 1005, 1002, 1010, 1005]",1005,"[Rain, Rain, Rain, Rain, Rain, Rain, Rain]",Rain,33.93,"11/18/2022, 13:07:14"
2,2,Bayeux,49.276462,-0.702474,"[1, 2, 3, 4, 5, 6, 7]","[9, 10, 3, 9, 9, 10, 13]",9,"[46, 74, 100, 100, 100, 100, 100]",89.0,"[67, 68, 85, 66, 83, 73, 74]",74.0,"[5.25, 8.81, 14.21, 13.84, 8.93, 13.31, 11.81]",11.0,"[1015, 1013, 989, 1003, 1000, 1011, 1005]",1005,"[Rain, Rain, Rain, Rain, Rain, Rain, Rain]",Rain,31.89,"11/18/2022, 13:07:14"
3,3,Le Havre,49.493898,0.107973,"[1, 2, 3, 4, 5, 6, 7]","[10, 10, 5, 10, 11, 11, 14]",10,"[66, 91, 100, 95, 83, 61, 36]",76.0,"[68, 68, 78, 62, 71, 83, 74]",72.0,"[5.95, 14.04, 13.04, 13.17, 10.42, 11.38, 10.35]",11.0,"[1014, 1012, 998, 1006, 1003, 1012, 1009]",1007,"[Rain, Rain, Rain, Rain, Rain, Rain, Rain]",Rain,30.12,"11/18/2022, 13:07:14"
4,4,Rouen,49.440459,1.093966,"[1, 2, 3, 4, 5, 6, 7]","[10, 9, 5, 9, 6, 10, 13]",8,"[37, 100, 100, 83, 98, 33, 26]",68.0,"[72, 75, 77, 70, 91, 80, 76]",77.0,"[5.22, 9.91, 7.11, 9.23, 6.84, 7.58, 7.33]",8.0,"[1014, 1011, 1000, 1006, 1002, 1014, 1010]",1008,"[Rain, Rain, Rain, Rain, Rain, Rain, Rain]",Rain,26.97,"11/18/2022, 13:07:14"


In [22]:
df_.columns

Index(['city_id', 'name', 'latitude', 'longitude', 'days', 'temperature',
       'mean_temp', 'proba_rain', 'mean_rain', 'humidity', 'mean_humidity',
       'wind_speed', 'mean_wind_speed', 'pressure', 'mean_pressure', 'weather',
       'mode_weather', 'score', 'date'],
      dtype='object')

In [23]:
def mode_weather(df):
    rain, clouds, clear = [], [], []
    for weather in df_['weather']:
        rain_count, clouds_count, clear_count = 0,0,0
        for el in weather:
            if el == 'Rain':
                rain_count += 1
            if el == 'Clouds':
                clouds_count += 1
            if el == 'Clear':
                clear_count += 1
        rain.append(rain_count)
        clouds.append(clouds_count)
        clear.append(clear_count)
    df['mode_rain'] = rain
    df['mode_clouds'] = clouds
    df['mode_clear'] = clear
    
    return df

In [24]:
df2 = df_.copy()
df2 = mode_weather(df2)
df2.columns

Index(['city_id', 'name', 'latitude', 'longitude', 'days', 'temperature',
       'mean_temp', 'proba_rain', 'mean_rain', 'humidity', 'mean_humidity',
       'wind_speed', 'mean_wind_speed', 'pressure', 'mean_pressure', 'weather',
       'mode_weather', 'score', 'date', 'mode_rain', 'mode_clouds',
       'mode_clear'],
      dtype='object')

In [25]:
def rank_df(df, coef_rain=0.2, coef_clouds=0.5, coef_clear=1, coef_weather=0.35, coef_temp=2, coef_himidity=0.2, coef_wind=0.9, coef_pressure=0.004):    
    # Calcul du rang Meteo : 0.2 * Pluvieux + 0.5 * Nuageux + 1 * Ciel_dégagé : on penalise le mauvais temps
    df['rank_weather'] = df['mode_rain']*coef_rain + df['mode_clouds']*coef_clouds + df['mode_clear']*coef_clear

    df['rank_weather'] = df['rank_weather'].rank(method='max').astype(int)
    df['rank_rain'] = df['mean_rain'].rank(method='min').astype(int)
    df['rank_temp'] = df['mean_temp'].rank(method='max').astype(int)
    df['rank_humi'] = df['mean_humidity'].rank(method='min').astype(int)
    df['rank_wind'] = df['mean_wind_speed'].rank(method='min').astype(int)
    df['rank_pres'] = df['mean_pressure'].rank(method='max').astype(int)


    # Calcul final du rang : 0.5 * rang_pluie + 0.35 * rang_meteo + 0.3 * rang_temperature + 0.9 * rang_humidité + 0.2 * rang_pression_atmospherique / nombre de rangs
    df['rank_'] = (df['rank_rain']*coef_rain + df['rank_weather']*coef_weather + df['rank_temp']*coef_temp + df['rank_humi']*coef_himidity + df['rank_wind']*coef_wind + df['rank_pres']*coef_pressure)/6
    df = df.sort_values(by='rank_').drop(df.columns[-7:], axis=1)
    df['rank'] = [i+1 for i in range(len(df))]
    df = df.reset_index().drop(['index'],axis=1)
    return df

In [26]:
df3 = df2.copy()
df3 = rank_df(df2)
df3.columns

Index(['city_id', 'name', 'latitude', 'longitude', 'days', 'temperature',
       'mean_temp', 'proba_rain', 'mean_rain', 'humidity', 'mean_humidity',
       'wind_speed', 'mean_wind_speed', 'pressure', 'mean_pressure', 'weather',
       'mode_weather', 'score', 'date', 'mode_rain', 'mode_clouds',
       'mode_clear', 'rank'],
      dtype='object')

In [27]:
df3.head()

,city_id,name,latitude,longitude,days,temperature,mean_temp,proba_rain,mean_rain,humidity,...,pressure,mean_pressure,weather,mode_weather,score,date,mode_rain,mode_clouds,mode_clear,rank
0,29,Ariege,42.945537,1.406554,"[1, 2, 3, 4, 5, 6, 7]","[0, 4, 7, -3, 5, 7, 9]",4,"[84, 76, 100, 100, 98, 76, 0]",76.0,"[94, 59, 72, 96, 84, 67, 67]",...,"[1016, 1020, 1007, 1015, 1013, 1018, 1015]",1014,"[Snow, Rain, Rain, Snow, Rain, Rain, Clouds]",Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
1,9,Chateau du Haut Koenigsbourg,48.249523,7.345492,"[1, 2, 3, 4, 5, 6, 7]","[4, 1, 5, 0, 2, 3, 6]",3,"[52, 100, 84, 100, 92, 88, 0]",74.0,"[76, 78, 74, 78, 74, 85, 62]",...,"[1011, 1016, 1009, 1005, 1009, 1020, 1019]",1012,"[Clouds, Rain, Rain, Rain, Rain, Rain, Clouds]",Rain,22.69,"11/18/2022, 13:07:14",5,2,0,2
2,7,Lille,50.636565,3.063528,"[1, 2, 3, 4, 5, 6, 7]","[3, 4, 6, 4, 4, 9, 11]",5,"[31, 100, 84, 94, 32, 99, 46]",69.0,"[85, 75, 77, 69, 78, 76, 63]",...,"[1012, 1009, 1003, 1002, 1004, 1014, 1011]",1007,"[Rain, Rain, Rain, Rain, Rain, Rain, Rain]",Rain,24.84,"11/18/2022, 13:07:14",7,0,0,3
3,13,Dijon,47.321581,5.041470,"[1, 2, 3, 4, 5, 6, 7]","[6, 4, 3, 4, 2, 8, 8]",5,"[35, 100, 100, 100, 88, 84, 0]",72.0,"[79, 74, 82, 69, 84, 76, 65]",...,"[1012, 1017, 1008, 1008, 1009, 1019, 1018]",1013,"[Rain, Rain, Rain, Rain, Rain, Rain, Clouds]",Rain,24.57,"11/18/2022, 13:07:14",6,1,0,4
4,14,Annecy,45.899235,6.128885,"[1, 2, 3, 4, 5, 6, 7]","[7, 7, 7, 3, 5, 7, 10]",6,"[54, 97, 99, 100, 78, 28, 0]",65.0,"[79, 76, 77, 98, 85, 87, 78]",...,"[1011, 1018, 1009, 1009, 1011, 1021, 1019]",1014,"[Rain, Rain, Rain, Snow, Rain, Rain, Clouds]",Rain,23.44,"11/18/2022, 13:07:14",5,1,0,5


In [28]:
df3['name'][:5]

0                          Ariege
1    Chateau du Haut Koenigsbourg
2                           Lille
3                           Dijon
4                          Annecy
Name: name, dtype: object

In [29]:
best_five_cities = df3['name'][:5].values.tolist()
best_five_cities

['Ariege', 'Chateau du Haut Koenigsbourg', 'Lille', 'Dijon', 'Annecy']

In [30]:
df3.to_csv('kayak2_final.csv', index=False)

In [31]:
df3 = pd.read_csv('kayak2_final.csv')
df3.head()

,city_id,name,latitude,longitude,days,temperature,mean_temp,proba_rain,mean_rain,humidity,...,pressure,mean_pressure,weather,mode_weather,score,date,mode_rain,mode_clouds,mode_clear,rank
0,29,Ariege,42.945537,1.406554,"[1, 2, 3, 4, 5, 6, 7]","[0, 4, 7, -3, 5, 7, 9]",4,"[84, 76, 100, 100, 98, 76, 0]",76.0,"[94, 59, 72, 96, 84, 67, 67]",...,"[1016, 1020, 1007, 1015, 1013, 1018, 1015]",1014,"['Snow', 'Rain', 'Rain', 'Snow', 'Rain', 'Rain...",Rain,24.21,"11/18/2022, 13:07:14",4,1,0,1
1,9,Chateau du Haut Koenigsbourg,48.249523,7.345492,"[1, 2, 3, 4, 5, 6, 7]","[4, 1, 5, 0, 2, 3, 6]",3,"[52, 100, 84, 100, 92, 88, 0]",74.0,"[76, 78, 74, 78, 74, 85, 62]",...,"[1011, 1016, 1009, 1005, 1009, 1020, 1019]",1012,"['Clouds', 'Rain', 'Rain', 'Rain', 'Rain', 'Ra...",Rain,22.69,"11/18/2022, 13:07:14",5,2,0,2
2,7,Lille,50.636565,3.063528,"[1, 2, 3, 4, 5, 6, 7]","[3, 4, 6, 4, 4, 9, 11]",5,"[31, 100, 84, 94, 32, 99, 46]",69.0,"[85, 75, 77, 69, 78, 76, 63]",...,"[1012, 1009, 1003, 1002, 1004, 1014, 1011]",1007,"['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Rain...",Rain,24.84,"11/18/2022, 13:07:14",7,0,0,3
3,13,Dijon,47.321581,5.041470,"[1, 2, 3, 4, 5, 6, 7]","[6, 4, 3, 4, 2, 8, 8]",5,"[35, 100, 100, 100, 88, 84, 0]",72.0,"[79, 74, 82, 69, 84, 76, 65]",...,"[1012, 1017, 1008, 1008, 1009, 1019, 1018]",1013,"['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Rain...",Rain,24.57,"11/18/2022, 13:07:14",6,1,0,4
4,14,Annecy,45.899235,6.128885,"[1, 2, 3, 4, 5, 6, 7]","[7, 7, 7, 3, 5, 7, 10]",6,"[54, 97, 99, 100, 78, 28, 0]",65.0,"[79, 76, 77, 98, 85, 87, 78]",...,"[1011, 1018, 1009, 1009, 1011, 1021, 1019]",1014,"['Rain', 'Rain', 'Rain', 'Snow', 'Rain', 'Rain...",Rain,23.44,"11/18/2022, 13:07:14",5,1,0,5
